In [2]:
import json

with open("./dataset/wikipedia_documents_ssm.json", "r") as f1, open("./dataset/wikipedia_documents_ssm1.json", "r") as f2:
    wiki1 = json.load(f1)
    print(len(wiki1))
    wiki2 = json.load(f2)
    print(len(wiki2))
    wiki3 = wiki1 + wiki2
    print(len(wiki3))
    print(wiki3[0])
    with open("./dataset/wikipedia_documents_ssm2.json", "w") as f:
        json.dump(wiki3, f)
    

63720
63720
127440
{'document_text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n* (a) 영속적인 국민\n* (b) 일정한 영토\n* (c) 정부\n* (d) 타국과의 관계 참여 자격.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다.  이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n\n위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.\n* 일반 국제 승인을 받은 195개 자주 국가.\n** 유엔 가입 국가 193개\n** 성좌의 명칭으로', 'masked_strings': '이 목록에 실린 국가 기준은 [MASK][MASK] [MASK][MASK][MASK][MASK] [MASK][MASK]을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다. * ([MASK]) 영속적인 [MASK] * (b) 일정한 영토 * (c) [MASK] * (d) 타국과의 관계 참여 자격. 특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 [MASK][MASK][MASK]오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, [MASK][MASK][MASK] 협약 기준만이 국가 지위의 충분한 자격이든 아니든, [MASK]의 견해

In [2]:
from datasets import load_dataset

from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    AutoConfig,
    DataCollatorForLanguageModeling,
)
import torch
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

import argparse
from omegaconf import OmegaConf

import random
import warnings
from collections.abc import Mapping
from dataclasses import dataclass
from random import randint
from collections import defaultdict
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
import torch

class MyDataCollatorWithPadding(DataCollatorWithPadding):
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        max_len = 0
        for i in features:
            if len(i["input_ids"]) > max_len:
                max_len = len(i["input_ids"])
        
        batch = defaultdict(list)
        for item in features:
            for k in item:
                item[k] = torch.Tensor(item[k])
                padding_len = max_len - item[k].size(0)
                if k == "input_ids":
                    item[k] = torch.cat((item[k], torch.tensor([self.tokenizer.pad_token_id] * padding_len)), dim=0)
                else:
                    item[k] = torch.cat((item[k], torch.tensor([0] * padding_len)), dim=0)
                batch[k].append(item[k])

        for k in batch:
            batch[k] = torch.stack(batch[k], dim=0)
            batch[k] = batch[k].to(torch.long)
        
        return batch

    
def load_eval_dataset(tokenizer, datasets):
    # preprocessing
    def prepare_validation_features(examples):
        # truncation과 padding(length가 짧을때만)을 통해 toknization을 진행하며, stride를 이용하여 overflow를 유지합니다.
        # 각 example들은 이전의 context와 조금씩 겹치게됩니다.
        
        final_examples = {}
        final_examples['input_ids'] = examples['inputs']
        final_examples['labels'] = examples['labels']
        
        final_examples['attention_mask'] = []
        for i in examples['inputs']:
            final_examples['attention_mask'].append([1]*len(i))
        
        return final_examples

    train_dataset = datasets["train"]

    # Validation Feature 생성
    train_dataset = train_dataset.map(
        prepare_validation_features,
        batched = True,
        remove_columns=['document_text', 'masked_strings', 'inputs', 'labels']
    )
    return train_dataset


def ssm_pretrain():
    model_name = 'klue/roberta-large'

    # set up tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    
    # label 없이 가져오기 위해서 load_predict_dataset 사용
    ### Refactoring 필요! ###
    dataset = load_dataset("json", data_files="./dataset/wikipedia_documents_ssm2.json")
    train_dataset = load_eval_dataset(tokenizer, dataset)

    # Pretrained model for MaskedLM training
    model_config = AutoConfig.from_pretrained(model_name)  # 모델 가중치 불러오기
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_name, config=model_config)
    model.resize_token_embeddings(len(tokenizer))
    model.parameters
    model.to(device)

    data_collator = MyDataCollatorWithPadding(tokenizer)

    # cuda out-of-memory 발생하여 fp16 = True 로 변경
    training_args = TrainingArguments(
        output_dir="./klue-roberta-pretrained",
        learning_rate=3e-05,
        num_train_epochs=2,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        save_steps=4000,
        save_total_limit=2,
        save_strategy="steps",
        logging_dir="./logs",
        logging_steps=100,
        fp16=True, # 16비트로 변환
        fp16_opt_level="O1",
        resume_from_checkpoint=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )

    trainer.train()
    model.save_pretrained("./klue-roberta-pretrained")  # pretrained_model save

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
abc = [1,1,1,1,1]
torch.Tensor(abc)

tensor([1., 1., 1., 1., 1.])

In [4]:
model_name = 'klue/roberta-large'

# set up tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# label 없이 가져오기 위해서 load_predict_dataset 사용
### Refactoring 필요! ###
dataset = load_dataset("json", data_files="./dataset/wikipedia_documents_ssm_qa2.json")
train_dataset = load_eval_dataset(tokenizer, dataset)

# Pretrained model for MaskedLM training
model_config = AutoConfig.from_pretrained(model_name)  # 모델 가중치 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AutoModelForMaskedLM.from_pretrained(model_name, config=model_config)
model = AutoModelForMaskedLM.from_pretrained('klue-roberta-pretrained3')
model.resize_token_embeddings(len(tokenizer))
model.parameters
model.to(device)

data_collator = MyDataCollatorWithPadding(tokenizer)

Using custom data configuration default-4123cfdacdd623e5
Found cached dataset json (/opt/ml/.cache/huggingface/datasets/json/default-4123cfdacdd623e5/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 513.76it/s]
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/json/default-4123cfdacdd623e5/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-d31b2cc0ae2b3444.arrow


In [5]:
# cuda out-of-memory 발생하여 fp16 = True 로 변경
training_args = TrainingArguments(
    output_dir="./klue-roberta-pretrained_qa2",
    learning_rate=3e-05,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=5000,
    save_total_limit=1,
    save_strategy="steps",
    logging_dir="./logs",
    logging_steps=200,
    fp16=True, # 16비트로 변환
    fp16_opt_level="O1",
    resume_from_checkpoint=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()
model.save_pretrained("./klue-roberta-pretrained_qa2")  # pretrained_model save

Using cuda_amp half precision backend
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6897
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8630
  Number of trainable parameters = 336690432
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: jstep750. Use `wandb login --relogin` to force relogin


Step,Training Loss
200,0.051500
400,0.048900
600,0.045800
800,0.045900
1000,0.031400
1200,0.023600
1400,0.023200
1600,0.024800
1800,0.019800
2000,0.012900


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./klue-roberta-pretrained_qa2/config.json
Model weights saved in ./klue-roberta-pretrained_qa2/pytorch_model.bin


In [4]:
!nvidia-smi

Wed Jan  4 04:45:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    36W / 250W |  28445MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------